In [1]:
#default_exp python_call_graph

In [2]:
#export

import csrgraph as cg
import pandas as pd
import numpy as np
from github_search import python_tokens, paperswithcode_tasks
from mlutil.feature_extraction import embeddings
from mlutil import prototype_selection
import mlutil
from mlutil.feature_extraction import embeddings 

import ast
import astunparse
import csrgraph
import nodevectors

import os
import itertools
import igraph
from sklearn import metrics
import gensim

from operator import itemgetter
import pickle

/home/kuba/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
%cd ..

/home/kuba/Projects/github_search


In [4]:
python_files_df = pd.read_csv('data/python_files.csv')

In [5]:
python_files_df

,owner,repo_name,file_path,content,sha
0,trangvu,ape-npi,run-tests.py,#!/usr/bin/env python3\nimport subprocess\nimp...,5364c4a113de40cb17e40b052fa012ad7daba2bd
1,trangvu,ape-npi,scripts/apply_bpe.py,#!/usr/bin/env python3\n# Author: Rico Sennric...,261dd0ffc2e10ee32ea99ca80c99cde50a6e7636
2,trangvu,ape-npi,scripts/concat-bpe.py,#!/usr/bin/env python3\n\nimport argparse\npar...,e6bc04ce4b16c0b91d00fc576ed5d261c711760f
3,trangvu,ape-npi,scripts/copy-model.py,#!/usr/bin/env python3\nimport argparse\nimpor...,56014b1516fd6df606d500b5efc190efbe8d5203
4,trangvu,ape-npi,scripts/coverage.py,#!/usr/bin/env python3\n\nimport argparse\nfro...,f008bc3a194503c2e0c68ea2d5389db81905f6bd
...,...,...,...,...,...
560178,KyeongmoonKim,sb,tools/convert_from_depre.py,# --------------------------------------------...,4ed7125b8568a0f9c60a3bf4670747ec4c497942
560179,KyeongmoonKim,sb,tools/demo.py,#!/usr/bin/env python\n\n# -------------------...,2bd89335df588e010bbb22370274dcbd04bcb407
560180,KyeongmoonKim,sb,tools/reval.py,#!/usr/bin/env python\n\n# -------------------...,612ae317fa5362b2b03b667cc2a8e17323d1a16f
560181,KyeongmoonKim,sb,tools/test_net.py,# --------------------------------------------...,c9cfc53f46f7f9d6c0885ffcad8727f86716ce22


In [6]:
paperswithcode_df, all_papers_df = paperswithcode_tasks.get_paperswithcode_dfs()

In [7]:
papers_with_repo_df = paperswithcode_tasks.get_papers_with_biggest_tasks_df(10000)

/home/kuba/Projects/github_search/github_search/paperswithcode_tasks.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  papers_with_repo_with_biggest_tasks_df['most_common_task'] = papers_with_repo_with_biggest_tasks_df['tasks'].apply(


In [8]:
papers_with_repo_df['repo_name'] = papers_with_repo_df['repo'].str.split("/").apply(lambda ps: ps[1])

In [9]:
papers_with_repo_df

,paper_url,arxiv_id,title,abstract,url_abs,url_pdf,proceeding,authors,tasks,date,...,mentioned_in_github,mentioned_in_paper,paper_arxiv_id,paper_title,paper_url_abs,paper_url_pdf,repo,repo_url,most_common_task,repo_name
2,https://paperswithcode.com/paper/procedural-3d...,2010.06411,Procedural 3D Terrain Generation using Generat...,Procedural 3D Terrain generation has become a ...,https://arxiv.org/abs/2010.06411v1,https://arxiv.org/pdf/2010.06411v1.pdf,None,"[Emmanouil Panagiotou, Eleni Charou]","[Colorization, Image-to-Image Translation]",2020-10-13,...,False,True,2010.06411,Procedural 3D Terrain Generation using Generat...,https://arxiv.org/abs/2010.06411v1,https://arxiv.org/pdf/2010.06411v1.pdf,Panagiotou/Procedural3DTerrain,https://github.com/Panagiotou/Procedural3DTerrain,Image-to-Image Translation,Procedural3DTerrain
6,https://paperswithcode.com/paper/multiscale-sp...,2003.11265,Multiscale Sparsifying Transform Learning for ...,The data-driven sparse methods such as synthes...,https://arxiv.org/abs/2003.11265v4,https://arxiv.org/pdf/2003.11265v4.pdf,None,"[Ashkan Abbasi, Amirhassan Monadjemi, Leyuan F...","[Denoising, Dictionary Learning, Image Denoising]",2020-03-25,...,True,False,2003.11265,Multiscale Sparsifying Transform Learning for ...,https://arxiv.org/abs/2003.11265v4,https://arxiv.org/pdf/2003.11265v4.pdf,ashkan-abbasi66/MTLD,https://github.com/ashkan-abbasi66/MTLD,Denoising,MTLD
7,https://paperswithcode.com/paper/word2vec-opti...,2003.11645,Word2Vec: Optimal Hyper-Parameters and Their I...,Word2Vec is a prominent model for natural lang...,https://arxiv.org/abs/2003.11645v2,https://arxiv.org/pdf/2003.11645v2.pdf,None,"[Tosin P. Adewumi, Foteini Liwicki, Marcus Liw...","[Named Entity Recognition, Sentiment Analysis]",2020-03-23,...,True,False,2003.11645,Word2Vec: Optimal Hyper-Parameters and Their I...,https://arxiv.org/abs/2003.11645v2,https://arxiv.org/pdf/2003.11645v2.pdf,phnk/D7047E,https://github.com/phnk/D7047E,Sentiment Analysis,D7047E
8,https://paperswithcode.com/paper/word2vec-opti...,2003.11645,Word2Vec: Optimal Hyper-Parameters and Their I...,Word2Vec is a prominent model for natural lang...,https://arxiv.org/abs/2003.11645v2,https://arxiv.org/pdf/2003.11645v2.pdf,None,"[Tosin P. Adewumi, Foteini Liwicki, Marcus Liw...","[Named Entity Recognition, Sentiment Analysis]",2020-03-23,...,False,True,2003.11645,Word2Vec: Optimal Hyper-Parameters and Their I...,https://arxiv.org/abs/2003.11645v2,https://arxiv.org/pdf/2003.11645v2.pdf,tosingithub/sdesk,https://github.com/tosingithub/sdesk,Sentiment Analysis,sdesk
9,https://paperswithcode.com/paper/evaluating-se...,1903.07377,Evaluating Sequence-to-Sequence Models for Han...,Encoder-decoder models have become an effectiv...,https://arxiv.org/abs/1903.07377v2,https://arxiv.org/pdf/1903.07377v2.pdf,None,"[Johannes Michael, Roger Labahn, Tobias Grünin...","[Image Captioning, Keyword Spotting, Language ...",2019-03-18,...,True,False,1903.07377,Evaluating Sequence-to-Sequence Models for Han...,https://arxiv.org/abs/1903.07377v2,https://arxiv.org/pdf/1903.07377v2.pdf,jamestjw/OCR,https://github.com/jamestjw/OCR,Language Modelling,OCR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74224,https://paperswithcode.com/paper/arbitrary-sty...,1703.06868,Arbitrary Style Transfer in Real-time with Ada...,Gatys et al. recently introduced a neural algo...,http://arxiv.org/abs/1703.06868v2,http://arxiv.org/pdf/1703.06868v2.pdf,ICCV 2017 10,"[Xun Huang, Serge Belongie]",[Style Transfer],2017-03-20,...,True,False,1703.06868,Arbitrary Style Transfer in Real-time with Ada...,http://arxiv.org/abs/1703.06868v2,http://arxiv.org/pdf/1703.06868v2.pdf,bethgelab/stylize-datasets,https://github.com/bethgelab/stylize-datasets,Style Transfer,stylize-datasets
74225,https://paperswithcode.com/paper/arbitrary-sty...,1703.06868,Arbitrary Style Transfer in Real-time with Ada...,Gatys et al. recently introduced a neural algo...,http://arxiv.org/abs/1703.06868v

In [10]:
python_files_df

,owner,repo_name,file_path,content,sha
0,trangvu,ape-npi,run-tests.py,#!/usr/bin/env python3\nimport subprocess\nimp...,5364c4a113de40cb17e40b052fa012ad7daba2bd
1,trangvu,ape-npi,scripts/apply_bpe.py,#!/usr/bin/env python3\n# Author: Rico Sennric...,261dd0ffc2e10ee32ea99ca80c99cde50a6e7636
2,trangvu,ape-npi,scripts/concat-bpe.py,#!/usr/bin/env python3\n\nimport argparse\npar...,e6bc04ce4b16c0b91d00fc576ed5d261c711760f
3,trangvu,ape-npi,scripts/copy-model.py,#!/usr/bin/env python3\nimport argparse\nimpor...,56014b1516fd6df606d500b5efc190efbe8d5203
4,trangvu,ape-npi,scripts/coverage.py,#!/usr/bin/env python3\n\nimport argparse\nfro...,f008bc3a194503c2e0c68ea2d5389db81905f6bd
...,...,...,...,...,...
560178,KyeongmoonKim,sb,tools/convert_from_depre.py,# --------------------------------------------...,4ed7125b8568a0f9c60a3bf4670747ec4c497942
560179,KyeongmoonKim,sb,tools/demo.py,#!/usr/bin/env python\n\n# -------------------...,2bd89335df588e010bbb22370274dcbd04bcb407
560180,KyeongmoonKim,sb,tools/reval.py,#!/usr/bin/env python\n\n# -------------------...,612ae317fa5362b2b03b667cc2a8e17323d1a16f
560181,KyeongmoonKim,sb,tools/test_net.py,# --------------------------------------------...,c9cfc53f46f7f9d6c0885ffcad8727f86716ce22


In [11]:
python_files_df = python_files_df.merge(papers_with_repo_df[['repo_name', 'tasks']], on=['repo_name'])

In [12]:
python_files_df

,owner,repo_name,file_path,content,sha,tasks
0,trangvu,ape-npi,run-tests.py,#!/usr/bin/env python3\nimport subprocess\nimp...,5364c4a113de40cb17e40b052fa012ad7daba2bd,"[Automatic Post-Editing, Machine Translation]"
1,trangvu,ape-npi,scripts/apply_bpe.py,#!/usr/bin/env python3\n# Author: Rico Sennric...,261dd0ffc2e10ee32ea99ca80c99cde50a6e7636,"[Automatic Post-Editing, Machine Translation]"
2,trangvu,ape-npi,scripts/concat-bpe.py,#!/usr/bin/env python3\n\nimport argparse\npar...,e6bc04ce4b16c0b91d00fc576ed5d261c711760f,"[Automatic Post-Editing, Machine Translation]"
3,trangvu,ape-npi,scripts/copy-model.py,#!/usr/bin/env python3\nimport argparse\nimpor...,56014b1516fd6df606d500b5efc190efbe8d5203,"[Automatic Post-Editing, Machine Translation]"
4,trangvu,ape-npi,scripts/coverage.py,#!/usr/bin/env python3\n\nimport argparse\nfro...,f008bc3a194503c2e0c68ea2d5389db81905f6bd,"[Automatic Post-Editing, Machine Translation]"
...,...,...,...,...,...,...
39906459,KyeongmoonKim,sb,tools/test_net.py,# --------------------------------------------...,c9cfc53f46f7f9d6c0885ffcad8727f86716ce22,[Object Detection]
39906460,KyeongmoonKim,sb,tools/trainval_net.py,# --------------------------------------------...,e7b4dab93b6e543402f25f53696e918f3e3d0b5a,[Object Detection]
39906461,KyeongmoonKim,sb,tools/trainval_net.py,# --------------------------------------------...,e7b4dab93b6e543402f25f53696e918f3e3d0b5a,[Object Detection]
39906462,KyeongmoonKim,sb,tools/trainval_net.py,# --------------------------------------------...,e7b4dab93b6e543402f25f53696e918f3e3d0b5a,"[Object Detection, Real-Time Object Detection,..."


In [13]:
example_file = python_files_df['content'].iloc[1]
print(example_file)

#!/usr/bin/env python3
# Author: Rico Sennrich

"""Use operations learned with learn_bpe.py to encode a new text.
The text will not be smaller, but use only a fixed vocabulary, with rare words
encoded as variable-length sequences of subword units.

Reference:
Rico Sennrich, Barry Haddow and Alexandra Birch (2016). Neural Machine Translation of Rare Words with Subword Units.
Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (ACL 2016). Berlin, Germany.
"""

import sys
import codecs
import argparse
import functools
from collections import defaultdict

class BPE(object):
    def encode(self, orig):
        """Encode word based on list of BPE merge operations, which are applied consecutively
        """
        word = tuple(orig) + ('</w>',)
        pairs = get_pairs(word)

        while True:
            bigram = min(pairs, key=lambda pair: self.bpe_codes.get(pair, float('inf')))
            if bigram not in self.bpe_codes:
                break
     

In [14]:
astunparse.unparse(ast.parse(example_file).body[-2])

"\n\ndef get_pairs(word):\n    'Return set of symbol pairs in a word.\\n\\n    word is represented as tuple of symbols (symbols being variable-length strings)\\n    '\n    pairs = set()\n    prev_char = word[0]\n    for char in word[1:]:\n        pairs.add((prev_char, char))\n        prev_char = char\n    return pairs\n"

In [15]:
fn_code = ast.parse(example_file).body[-2]

In [16]:
astunparse.unparse(fn_code.body[3])

'\nfor char in word[1:]:\n    pairs.add((prev_char, char))\n    prev_char = char\n'

In [17]:
%load_ext autoreload

In [18]:
code = """
def foo():
    bar()
"""

In [19]:
function_ast = (ast.parse(code).body[0])

In [20]:
ast.parse(code).body[0].name

'foo'

In [21]:
exprs = ast.parse(code).body[0].body

In [22]:
expr = exprs[0]

In [23]:
expr_ast = ast.parse("word = tuple(orig) + ''").body[0]

In [24]:
expr_ast.value

In [25]:
#export

def get_calls_from_expr_or_assign(expr):
    try:
        if type(expr) is ast.Name:
            return [expr.id]
        if type(expr) is ast.Call:
            return get_calls_from_expr_or_assign(expr.func) + get_calls_from_expr_or_assign(expr.args)
        elif type(expr) is ast.Attribute:
            return [expr.attr]
        elif type(expr) is ast.BinOp:
            return get_calls_from_expr_or_assign(expr.left) + get_calls_from_expr_or_assign(expr.right)
        elif type(expr) is ast.Expr:
            return get_calls_from_expr_or_assign(expr.value)
        elif type(expr) is ast.Assign:
            return get_calls_from_expr_or_assign(expr.value)
        elif type(expr) is ast.While or type(expr) is ast.If:
            return (
                get_calls_from_expr_or_assign(expr.test) +
                get_calls_from_expr_or_assign(expr.orelse) +
                get_calls_from_expr_or_assign(expr.body)
            )
        elif type(expr) is ast.For:
            return (
                get_calls_from_expr_or_assign(expr.target) +
                get_calls_from_expr_or_assign(expr.body) +
                get_calls_from_expr_or_assign(expr.iter)
            )
        elif type(expr) is ast.Try:
            return get_calls_from_expr_or_assign(expr.body)
        elif type(expr) is list:
            return [res for subexpr in expr for res in get_calls_from_expr_or_assign(subexpr)]
        else:
            return []
    except Exception as e:
        print(str(astunparse.unparse(expr)), ' ', str(e))

In [26]:
expr_ast.value.right

In [27]:
expr_ast.value

In [28]:
astunparse.unparse(expr_ast.value)

"(tuple(orig) + '')\n"

In [29]:
expr_ast.value.left.func.id

'tuple'

In [30]:
get_calls_from_expr_or_assign(expr_ast)

['tuple', 'orig']

In [31]:
encode_code = """def encode(self, orig):
    word = tuple(orig) + ('</w>',)
    pairs = get_pairs(word)

    while True:
        bigram = min(pairs, key=lambda pair: self.bpe_codes.get(pair, float('inf')))
        if bigram not in self.bpe_codes:
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
                new_word.append(first + second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # don't print end-of-word symbols
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>', ''),)

    return word
"""

In [32]:
encode_ast = ast.parse(encode_code)

In [33]:
assign_body = encode_ast.body[0].body[0]

In [34]:
assign_body.value

In [35]:
astunparse.unparse(assign_body.value)

"(tuple(orig) + ('</w>',))\n"

In [36]:
class_def = ast.parse(code).body[0]

In [37]:
class_def.body

In [38]:
#export

def zip_dicts(dicts):
    res = {}
    for d in dicts:
        res = {**res, **d}
    return res


def get_ast_function_calls(code_ast, calls={}):
    if type(code_ast) is ast.FunctionDef:
        return {code_ast.name: frozenset(
            sum([get_calls_from_expr_or_assign(expr) for expr in code_ast.body], [])
        )}
    elif type(code_ast) is ast.ClassDef:
        return {code_ast.name: frozenset(
            sum([get_calls_from_expr_or_assign(expr) for expr in code_ast.body], [])
        )}
    elif hasattr(code_ast, 'body'):
        return zip_dicts([get_ast_function_calls(item) for item in code_ast.body])
    else: return {}
    
    
def get_function_calls(code):
    code_ast = ast.parse(code)
    return get_ast_function_calls(code_ast)

In [39]:
e = ast.parse(
"new_word.extend(word[i:j])"
).body[0]

In [40]:
get_ast_function_calls(encode_ast.body[0])

{'encode': frozenset({'append',
            'bigram',
            'endswith',
            'extend',
            'first',
            'get_pairs',
            'i',
            'index',
            'j',
            'min',
            'new_word',
            'orig',
            'pairs',
            'second',
            'tuple',
            'word'})}

In [41]:
get_ast_function_calls(ast.parse(encode_code).body[0])

{'encode': frozenset({'append',
            'bigram',
            'endswith',
            'extend',
            'first',
            'get_pairs',
            'i',
            'index',
            'j',
            'min',
            'new_word',
            'orig',
            'pairs',
            'second',
            'tuple',
            'word'})}

In [42]:
get_function_calls(code)

{'foo': frozenset({'bar'})}

In [43]:
assert get_function_calls(code) == {"foo": frozenset(["bar"])}

In [44]:
items = ast.parse(code).body

In [45]:
items

In [46]:
code = """
class BPE(object):
    def encode(self, orig):
        word = tuple(orig) + ('</w>',)
        pairs = get_pairs(word)

        while True:
            bigram = min(pairs, key=lambda pair: self.bpe_codes.get(pair, float('inf')))
            if bigram not in self.bpe_codes:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
                    new_word.append(first + second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)

        # don't print end-of-word symbols
        if word[-1] == '</w>':
            word = word[:-1]
        elif word[-1].endswith('</w>'):
            word = word[:-1] + (word[-1].replace('</w>', ''),)

        return word

    def __init__(self, codes, separator='@@'):
        self.encode = functools.lru_cache(maxsize=65536)(self.encode)
        self.bpe_codes = [tuple(item.split()) for item in codes]
        # some hacking to deal with duplicates (only consider first instance)
        self.bpe_codes = dict([(code,i) for (i,code) in reversed(list(enumerate(self.bpe_codes)))])

        self.separator = separator

    def segment(self, sentence):
        output = []
        for word in sentence.split():
            new_word = self.encode(word)

            for item in new_word[:-1]:
                output.append(item + self.separator)
            output.append(new_word[-1])

        return ' '.join(output)
"""

In [47]:
get_function_calls(example_file)

{'BPE': frozenset(),
 'create_parser': frozenset({'ArgumentParser', 'add_argument'}),
 'get_pairs': frozenset({'add', 'char', 'set'})}

In [48]:
items = ast.parse("a.append(1)").body
expr = items[0].value

In [49]:
%%time
python_files_df['clean_content'] = python_files_df['content'].str.replace("429: Too Many Requests", "")

CPU times: user 51.1 s, sys: 242 ms, total: 51.3 s
Wall time: 51.3 s


In [50]:
python_files_df['clean_content'] = python_files_df['clean_content'].apply(lambda s: np.nan if s == "" else s)

In [51]:
sample_df = python_files_df.dropna().groupby('repo_name').apply(lambda df: df.iloc[:min(df.shape[0], 100)]).reset_index(drop=True)

In [52]:
example_files = sample_df['content']

In [53]:
example_files

0        # Copyright 2019-2020 Stanislav Pidhorskyi\n#\...
1        # Copyright 2019-2020 Stanislav Pidhorskyi\n#\...
2        # Copyright 2019-2020 Stanislav Pidhorskyi\n#\...
3        # Copyright 2019 Stanislav Pidhorskyi\n#\n# Li...
4        # Copyright 2019-2020 Stanislav Pidhorskyi\n# ...
                               ...                        
89233    import argparse\nimport re\nimport os\nimport ...
89234    import json\nimport xml.etree.ElementTree as E...
89235    import json\nimport xml.etree.ElementTree as E...
89236    import json\nimport xml.etree.ElementTree as E...
89237    import argparse\nimport json\nimport numpy as ...
Name: content, Length: 89238, dtype: object

In [54]:
example_filenames = sample_df['file_path'].apply(os.path.basename).str.replace('.py', '')

In [55]:
example_filenames.head()

0      checkpointer
1               ppl
2             model
3    model_separate
4               net
Name: file_path, dtype: object

In [200]:
def get_repo_task_edges(python_files_df):
    task_exploded_df = python_files_df[['repo_name', 'tasks']].explode('tasks')
    task_exploded_df['tasks'] = task_exploded_df['tasks'].str.replace("2D ", "").str.replace("3D ", "").str.replace("4D ", "").str.replace("6D ", "")
    return task_exploded_df.groupby("tasks").apply(lambda df: {df['tasks'].iloc[0]: frozenset(df['repo_name'].values)}).tolist()

In [201]:
repo_task_edges = get_repo_task_edges(sample_df)

In [202]:
repo_task_edges

[{'': frozenset({'deepmind-research', 'pytorch-image-models'})},
 {'AMR Parsing': frozenset({'stog'})},
 {'AMR-to-Text Generation': frozenset({'OpenNMT-py'})},
 {'Abstractive Text Summarization': frozenset({'BERT_text_summarisation',
             'Chatbot',
             'CopyCat-abstractive-opinion-summarizer',
             'Pointer_Transformer_Generator',
             'bottom-up-summary',
             'fairseq',
             'google-research',
             'onnxt5',
             'politicalsynthesis',
             'qags',
             'shmoop-corpus',
             'swap-net',
             'teaser_generate',
             'tensor2tensor',
             'transformers'})},
 {'Abuse Detection': frozenset({'multi-task-offensive-language-detection'})},
 {'Abusive Language': frozenset({'emo2vec_wassa_paper',
             'hate-speech-detection'})},
 {'Accented Speech Recognition': frozenset({'DeepSpeech',
             'STT',
             'audio2score',
             'models'})},
 {'Acoustic Scen

In [203]:
#export

def try_run(f): 
    def _maybe_failed_f(args):
        try:
            return f(args)
        except:
            return None
    return _maybe_failed_f

In [204]:
python_files_df[:100]

,owner,repo_name,file_path,content,sha,tasks,clean_content
0,trangvu,ape-npi,run-tests.py,#!/usr/bin/env python3\nimport subprocess\nimp...,5364c4a113de40cb17e40b052fa012ad7daba2bd,"[Automatic Post-Editing, Machine Translation]",#!/usr/bin/env python3\nimport subprocess\nimp...
1,trangvu,ape-npi,scripts/apply_bpe.py,#!/usr/bin/env python3\n# Author: Rico Sennric...,261dd0ffc2e10ee32ea99ca80c99cde50a6e7636,"[Automatic Post-Editing, Machine Translation]",#!/usr/bin/env python3\n# Author: Rico Sennric...
2,trangvu,ape-npi,scripts/concat-bpe.py,#!/usr/bin/env python3\n\nimport argparse\npar...,e6bc04ce4b16c0b91d00fc576ed5d261c711760f,"[Automatic Post-Editing, Machine Translation]",#!/usr/bin/env python3\n\nimport argparse\npar...
3,trangvu,ape-npi,scripts/copy-model.py,#!/usr/bin/env python3\nimport argparse\nimpor...,56014b1516fd6df606d500b5efc190efbe8d5203,"[Automatic Post-Editing, Machine Translation]",#!/usr/bin/env python3\nimport argparse\nimpor...
4,trangvu,ape-npi,scripts/coverage.py,#!/usr/bin/env python3\n\nimport argparse\nfro...,f008bc3a194503c2e0c68ea2d5389db81905f6bd,"[Automatic Post-Editing, Machine Translation]",#!/usr/bin/env python3\n\nimport argparse\nfro...
...,...,...,...,...,...,...,...
95,harryhan618,LaneNet,model.py,import torch\nimport torch.nn as nn\nimport to...,f97338efc97ad3a902c0db0c5084fdc0247d136e,"[Instance Segmentation, Lane Detection, Semant...",import torch\nimport torch.nn as nn\nimport to...
96,harryhan618,LaneNet,model.py,import torch\nimport torch.nn as nn\nimport to...,f97338efc97ad3a902c0db0c5084fdc0247d136e,"[Instance Segmentation, Lane Detection, Metric...",import torch\nimport torch.nn as nn\nimport to...
97,harryhan618,LaneNet,test_tusimple.py,import os\nimport json\nimport argparse\n\nimp...,c9e8ed9c0e11dc8c1e4ac3cce422ccb370bb58e6,"[Instance Segmentation, Lane Detection, Semant...",import os\nimport json\nimport argparse\n\nimp...
98,harryhan618,LaneNet,test_tusimple.py,import os\nimport json\nimport argparse\n\nimp...,c9e8ed9c0e11dc8c1e4ac3cce422ccb370bb58e6,"[Instance Segmentation, Lane Detection, Semant...",import os\nimport json\nimport argparse\n\nimp...


In [205]:
%%time
example_function_edges = example_files.apply(try_run(get_function_calls)).dropna()

CPU times: user 1min 47s, sys: 59.4 ms, total: 1min 47s
Wall time: 1min 47s


In [206]:
example_function_edges = example_function_edges.to_list()

In [207]:
#export


def get_upper_level_edges(upper_level_names, lower_level_edges):
    return [
        {
            file_name: 
            frozenset(
                [
                    function 
                    for function in lower_level_edges.keys()
                ])
        }
        for (file_name, lower_level_edges) in zip(upper_level_names, lower_level_edges)
        if type(lower_level_edges) is dict
    ]


def make_records(function_edges):
    return [{
        'calling_function': calling_fn, 
        'called_function': called_fn
    }
        for calling_fn in function_edges.keys()
        for called_fn in function_edges[calling_fn]
    ]

In [208]:
#export


example_node_names = [
    'train', 'fit', 'test', '__init__', 'DistilBertForMaskedLM',
    'tensorflow', 'transformers', 'mmdetection', 'Recommenders-movielens',
    'hate-speech-detection', 'variational-dropout-sparsifies-dnn']


def get_nodes(call_graph, node_names):
    selected_nodes = call_graph.names[call_graph.names.isin(node_names)]
    return selected_nodes


def get_node_embedding_model_and_results(node_embeddings, call_graph, example_nodes, selected_names=None, topk=10):
    if selected_names is None:
        selected_nodes = call_graph.names
    else:
        selected_nodes = get_node_indices(call_graph, selected_names)
        
    
    results = []
    for (i, node_name) in enumerate(example_nodes.values):
        distances = metrics.euclidean_distances(node_embeddings[example_nodes.index[[i]]], node_embeddings) 
        results.append((call_graph.names[np.argsort(distances)[0,:topk]].values))
    results_df = pd.DataFrame(results)
    results_df.index =  example_nodes.values
    return results_df.T

In [209]:
filename_edges = get_upper_level_edges(example_filenames, example_function_edges)

In [210]:
repo_edges_dict = example_filenames.groupby(sample_df['repo_name']).agg(lambda args: frozenset(args)).to_dict()

In [211]:
raw_repo_edges = [{k: repo_edges_dict[k]} for k in repo_edges_dict]

In [212]:
repo_edges_dict

{'-Adversarial-Latent-Autoencoders': frozenset({'checkpointer',
            'classifier',
            'download_all',
            'extract_attributes',
            'find_principal_directions',
            'generate_images',
            'model',
            'model_separate',
            'net',
            'ppl',
            'registry',
            'scheduler',
            'stylemix',
            'tracker',
            'train_alae',
            'train_alae_separate',
            'utils'}),
 '110kDBRD': frozenset({'gather_urls'}),
 '2020-CBMS-DoubleU-Net': frozenset({'data',
            'metrics',
            'model',
            'predict',
            'train',
            'utils'}),
 '2020-NeurIPS-CLEARER': frozenset({'architect',
            'datasets',
            'pytorch_ssim',
            'search_space',
            'test_model',
            'train_model',
            'train_search',
            'utils'}),
 '2020-Ultramicro-fast': frozenset({'add_patch',
            'basis_analysis'

In [213]:
root_edges = [{
    '<ROOT>': [repo for edges in raw_repo_edges for repo in repo_edges_dict.keys()]
    }
]

In [214]:
repo_edges = root_edges + raw_repo_edges

In [215]:
repo_task_edges

[{'': frozenset({'deepmind-research', 'pytorch-image-models'})},
 {'AMR Parsing': frozenset({'stog'})},
 {'AMR-to-Text Generation': frozenset({'OpenNMT-py'})},
 {'Abstractive Text Summarization': frozenset({'BERT_text_summarisation',
             'Chatbot',
             'CopyCat-abstractive-opinion-summarizer',
             'Pointer_Transformer_Generator',
             'bottom-up-summary',
             'fairseq',
             'google-research',
             'onnxt5',
             'politicalsynthesis',
             'qags',
             'shmoop-corpus',
             'swap-net',
             'teaser_generate',
             'tensor2tensor',
             'transformers'})},
 {'Abuse Detection': frozenset({'multi-task-offensive-language-detection'})},
 {'Abusive Language': frozenset({'emo2vec_wassa_paper',
             'hate-speech-detection'})},
 {'Accented Speech Recognition': frozenset({'DeepSpeech',
             'STT',
             'audio2score',
             'models'})},
 {'Acoustic Scen

In [195]:
all_edges = root_edges + repo_task_edges + raw_repo_edges +  filename_edges + example_function_edges

In [220]:
%%time
records  = [ record
    for edges in all_edges 
    for record in make_records(edges)
]

CPU times: user 936 ms, sys: 432 ms, total: 1.37 s
Wall time: 1.37 s


In [221]:
len(set(map(itemgetter('calling_function'), records)))

66813

In [222]:
call_records_df = pd.DataFrame.from_records(records)
call_records_df.columns = ['source', 'destination']

In [223]:
transitive_closure_df = call_records_df

In [224]:
call_records_df = call_records_df[
    (call_records_df['source'].apply(len) > 0) &
    (call_records_df['source'] != 'null') &
    (call_records_df['destination'].apply(len) > 0) &
    (call_records_df['source'] != call_records_df['destination'])
]

In [225]:
call_records_df = call_records_df.drop_duplicates()

In [226]:
call_records_df.to_csv('data/call_records.csv', index=None)

In [227]:
relations = list(call_records_df.iloc[:,[1,0]].itertuples(index=False, name=None))

In [228]:
call_records_df

,source,destination
0,<ROOT>,-Adversarial-Latent-Autoencoders
1,<ROOT>,110kDBRD
2,<ROOT>,2020-CBMS-DoubleU-Net
3,<ROOT>,2020-NeurIPS-CLEARER
4,<ROOT>,2020-Ultramicro-fast
...,...,...
10035941,test,sort_id
10035943,test,hit_count
10035944,test,index_list
10035946,test,current_len


In [229]:
call_records_df.to_csv('data/function_calls.csv', index=False, header=False)

In [230]:
call_records_df.iloc[:132641]

,source,destination
0,<ROOT>,-Adversarial-Latent-Autoencoders
1,<ROOT>,110kDBRD
2,<ROOT>,2020-CBMS-DoubleU-Net
3,<ROOT>,2020-NeurIPS-CLEARER
4,<ROOT>,2020-Ultramicro-fast
...,...,...
8027097,distRL,pricenorm3d
8027098,distRL,get_type_list
8027099,distRL,panel2array
8027100,distRL,count_periods


In [231]:
call_graph = cg.read_edgelist('data/function_calls.csv', sep=',')

In [232]:
example_nodes = get_nodes(call_graph, example_node_names)

In [233]:
pickle.dump(call_graph, open('call_graph.pkl', 'wb'))

In [234]:
vertices = list(set(call_records_df['source'].unique()).union(call_records_df['destination'].unique()))

In [235]:
edges = [
    (source, destination)
    for (source, destination) in zip(
        call_records_df['source'].values,
        call_records_df['destination'].values
    )
]

In [236]:
graph = igraph.Graph()
graph.add_vertices(vertices)
graph.add_edges(edges)

In [237]:
pickle.dump(graph, open("data/call_igraph.pkl", 'wb'))

In [238]:
call_records_df.shape

(844727, 2)

In [239]:
type(call_graph)

csrgraph.graph.csrgraph

In [240]:
len(call_graph.dst)

844727

In [241]:
call_graph.mat

<198514x198514 sparse matrix of type '<class 'numpy.float64'>'
	with 844727 stored elements in Compressed Sparse Row format>

In [242]:
repo_edges

[{'<ROOT>': ['-Adversarial-Latent-Autoencoders',
   '110kDBRD',
   '2020-CBMS-DoubleU-Net',
   '2020-NeurIPS-CLEARER',
   '2020-Ultramicro-fast',
   '2020a_IMT_SSH_mapping_NATL60',
   '2D-3D_Multitask_Deep_Learning',
   '2D-TAN',
   '3D-BoundingBox',
   '3D-Model-Reconstruction',
   '3D-RU-Net',
   '3D-human-pose-estimation',
   '3D60',
   '3DIndoor-SceneGraphNet',
   '3DKeypoints-DA',
   '3D_DenseSeg',
   '3D_Pose_Estimation',
   '3dflat',
   '4501Project',
   '558-project',
   'A-Deep-Neural-Solution-To-Document-Level-Novelty-Detection-COLING-2018-',
   'AADM',
   'AAE',
   'AAL-unbiased-authentication',
   'ABAW2020TNT',
   'ABCD-MICCAI2019',
   'ABD-NMT',
   'ACDNE',
   'ACE',
   'ACL2019-dp-cross-domain',
   'AComPianist',
   'ADDA',
   'ADEPT-Model-Release',
   'ADMMiRNN',
   'AE_Detector',
   'AFET',
   'AGRL.pytorch',
   'AGRR-2019',
   'AI-projects',
   'AIF',
   'AIF-CNN',
   'AIMS',
   'AIP',
   'AMOS',
   'ANPR-with-Yolov4',
   'APES',
   'ARC-Challenge',
   'ARG',
   'ARTN

In [243]:
len(repo_edges_dict)

2807

## nodevectors & csrgraph

In [259]:
example_repo_node_names = [
    'fairseq', 'mmdetection', 'Recommenders-movielens',
]
example_node_names = [
    'train', 'fit', 'test', '__init__', 'DistilBertForMaskedLM',
    'tensorflow', 
    'hate-speech-detection', 'variational-dropout-sparsifies-dnn']



def get_nodes(call_graph, node_names):
    selected_nodes = call_graph.names[call_graph.names.isin(node_names)]
    return selected_nodes


example_nodes = get_nodes(call_graph, example_node_names)
example_repo_nodes = get_nodes(call_graph, example_repo_node_names)


def get_node_embedding_model_and_results(node_embeddings, call_graph, example_nodes, selected_names=None, topk=10):
    if selected_names is None:
        selected_nodes = call_graph.names
    else:
        selected_nodes = get_node_indices(call_graph, selected_names)
        
    
    results = []
    for (i, node_name) in enumerate(example_nodes.values):
        distances = metrics.euclidean_distances(node_embeddings[example_nodes.index[[i]]], node_embeddings) 
        results.append((call_graph.names[np.argsort(distances)[0,:topk]].values))
    results_df = pd.DataFrame(results)
    results_df.index =  example_nodes.values
    return results_df.T

In [260]:
call_records_df[call_records_df['source'] == 'fairseq']

,source,destination
7916228,fairseq,conf
7916229,fairseq,__init__


In [261]:
call_records_df[call_records_df['destination'] == 'fairseq']

,source,destination
1878,<ROOT>,fairseq
7879272,Abstractive Text Summarization,fairseq
7879611,Automatic Post-Editing,fairseq
7879954,Common Sense Reasoning,fairseq
7880248,Cross-Lingual Natural Language Inference,fairseq
7880712,Dialogue Generation,fairseq
7881084,End-To-End Speech Recognition,fairseq
7881786,Image Captioning,fairseq
7881934,Image Classification,fairseq
7882960,Language Modelling,fairseq


In [262]:
call_records_df[call_records_df['source'] == '<ROOT>']

,source,destination
0,<ROOT>,-Adversarial-Latent-Autoencoders
1,<ROOT>,110kDBRD
2,<ROOT>,2020-CBMS-DoubleU-Net
3,<ROOT>,2020-NeurIPS-CLEARER
4,<ROOT>,2020-Ultramicro-fast
...,...,...
2802,<ROOT>,yolov3_jetbot
2803,<ROOT>,yolov4_train
2804,<ROOT>,youtube
2805,<ROOT>,youtube-8m


In [263]:
call_graph.mat.diagonal().sum()

0.0

%%time
ggvec_model = nodevectors.GGVec(n_components=100, max_epoch=1000)
ggvec_embeddings = ggvec_model.fit_transform(call_graph)

In [264]:
%%time
prone_model = nodevectors.ProNE(n_components=100, step=100)
prone_embeddings = prone_model.fit_transform(call_graph)

CPU times: user 2min 25s, sys: 1min 24s, total: 3min 50s
Wall time: 45.3 s


get_node_embedding_model_and_results(ggvec_embeddings, call_graph, example_repo_nodes)

In [265]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_repo_nodes)

,Recommenders-movielens,fairseq,mmdetection
0,Recommenders-movielens,fairseq,mmdetection
1,predict2,autopool,scopes
2,PointNet-,test_modelling_funcs,StandAloneSelfAttentionCifar10
3,RecoverSAT,plotting_with_arrows,ResBlockCifar10
4,update_q,from_hyperparameters,choreography
5,sdcar,product_kernel,h36m_train_vae
6,VAE-CF,fabolas,DenseBlockCifar10
7,samplefocus2,OpenMatch,ConvAM
8,round_through,ElapsedTime,spacy
9,ant_data_collect,pe-decipher-toolkit,charades-algorithms


In [266]:
from gensim.models import KeyedVectors
prone_kv = KeyedVectors(prone_embeddings.shape[1])
prone_kv.add(call_graph.names, prone_embeddings)

In [267]:
prone_kv.save("data/prone_embeddings.bin")

In [106]:
prone_kv.most_similar("tensorflow")

/home/kuba/.local/lib/python3.8/site-packages/gensim/models/keyedvectors.py:2389: RuntimeWarning: invalid value encountered in true_divide
  return (m / dist).astype(REAL)


[('c_to_dir', 1.0),
 ('gen_reg', 1.0),
 ('compile_java', 1.0),
 ('configure_file', 1.0),
 ('Gaussian_estimators', 1.0),
 ('util_causality', 1.0),
 ('coverage-info', 1.0),
 ('treatment_outcome_predictor', 1.0),
 ('gen3_reg', 1.0),
 ('gen_protos', 1.0)]

In [107]:
[k for k in prone_kv.vocab.keys() if 'allenn' in k]

['allennlp_cmd', 'allennlp_coref', 'use_allennlp']

In [108]:
len(list(repo_edges[0].values())[0])

7879249

In [109]:
list(repo_edges[0].values())[0]

['-Adversarial-Latent-Autoencoders',
 '110kDBRD',
 '2020-CBMS-DoubleU-Net',
 '2020-NeurIPS-CLEARER',
 '2020-Ultramicro-fast',
 '2020a_IMT_SSH_mapping_NATL60',
 '2D-3D_Multitask_Deep_Learning',
 '2D-TAN',
 '3D-BoundingBox',
 '3D-Model-Reconstruction',
 '3D-RU-Net',
 '3D-human-pose-estimation',
 '3D60',
 '3DIndoor-SceneGraphNet',
 '3DKeypoints-DA',
 '3D_DenseSeg',
 '3D_Pose_Estimation',
 '3dflat',
 '4501Project',
 '558-project',
 'A-Deep-Neural-Solution-To-Document-Level-Novelty-Detection-COLING-2018-',
 'AADM',
 'AAE',
 'AAL-unbiased-authentication',
 'ABAW2020TNT',
 'ABCD-MICCAI2019',
 'ABD-NMT',
 'ACDNE',
 'ACE',
 'ACL2019-dp-cross-domain',
 'AComPianist',
 'ADDA',
 'ADEPT-Model-Release',
 'ADMMiRNN',
 'AE_Detector',
 'AFET',
 'AGRL.pytorch',
 'AGRR-2019',
 'AI-projects',
 'AIF',
 'AIF-CNN',
 'AIMS',
 'AIP',
 'AMOS',
 'ANPR-with-Yolov4',
 'APES',
 'ARC-Challenge',
 'ARG',
 'ARTNet',
 'AReS-MaRS',
 'ASG',
 'ASX300',
 'AT4ChineseNER',
 'ATDA',
 'ATL_Python',
 'ATVGnet',
 'AUTOKERAS',
 '

In [110]:
call_graph.names[call_graph.names.str.contains('allennlp')]

46007       allennlp_cmd
46008     allennlp_coref
188702      use_allennlp
dtype: object

In [111]:
[k for item in repo_edges for (k, v) in item.items() if 'basic_allennlp' in v]

[]

In [112]:
prone_kv.most_similar("num_block_x")

[('norm_quat', nan),
 ('norm_range', nan),
 ('norm_ones', nan),
 ('norm_output_rois', nan),
 ('norm_params', nan),
 ('norm_points', nan),
 ('norm_points_with_T', nan),
 ('norm_predicted', nan),
 ('norm_prep', nan),
 ('norm_proposal_list', nan)]

In [113]:
with open('prone_closest_repos.md', 'w') as f:
    f.write(get_node_embedding_model_and_results(prone_embeddings, call_graph, example_repo_nodes).to_markdown())

In [114]:
with open('prone_closest_nodes.md', 'w') as f:
    f.write(get_node_embedding_model_and_results(prone_embeddings, call_graph, example_nodes).to_markdown())

In [115]:
!cat prone_closest_nodes.md

|    | DistilBertForMaskedLM           | __init__   | fit                              | hate-speech-detection      | tensorflow                  | test       | train       | variational-dropout-sparsifies-dnn                   |
|---:|:--------------------------------|:-----------|:---------------------------------|:---------------------------|:----------------------------|:-----------|:------------|:-----------------------------------------------------|
|  0 | gluLookAt                       | __init__   | fit                              | loss_opr                   | util_causality              | test       | train       | variational-dropout-sparsifies-dnn                   |
|  1 | make_2way_boxplot               | train      | validate                         | tensor_ops                 | tensorflow                  | evaluate   | __init__    | virtual                                              |
|  2 | make_EuclideanClusterExtraction | main       | train_network_one_epoch   

In [116]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_nodes).to_csv(open('prone_closest_nodes.md', 'w'))

In [117]:
get_node_embedding_model_and_results(ggvec_embeddings, call_graph, example_nodes)

NameError: name 'ggvec_embeddings' is not defined

In [ ]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_nodes)

In [ ]:
from gensim.models import poincare

In [ ]:
example_edges = list(call_records_df.itertuples(index=False, name=None))

In [ ]:
call_records_df.shape

In [ ]:
model = poincare.PoincareModel(example_edges, negative=2)

In [ ]:
%%time
model.train(epochs=2, batch_size=64)

In [ ]:
model.kv.descendants("tensorflow")

# Hyperbolic embeddings

In [ ]:
import torch

In [ ]:
%load_ext autoreload

In [ ]:
torch.model

## TODO

- expore call graph
- better call graph
- Node2

Exploring call graph

what is the problem?

Graph is not even connected

In [ ]:
i = 2